# User Access Review Script

User access testing is one of the most common tasks IT auditors will undertake during their career. Checking that access to a given system is of critical importance, and auditors will spend a lot of time performing different tests on user lists provided by those being audited, or pulled using tools like PowerShell or other user management tools. 

The tests themselves are relatively straight forward and often include tasks such as checking for terminated users with enabled accounts, accounts that have been newly provisioned, and appropriateness of a person's permissions given their role within an organization. However, frequently user lists lack adequate information on their own to complete these tests. Often the solution is to take a user listing and then compare it to an HR report that contains information such as hire/termination dates and a user's role within the company. This information can be easily linked together via Excel using vlookups, or manually searching for a user within an HR report and then copying over related information. These tasks are simple enough, but often take time and are prone to user error due to their manual nature.

This example script aims to somewhat automate the process by using two imputs: 1) a user list, and 2) an HR report (or similar). The script performs a simple join from one to the other via a primary key and then creates a .xlsx file containing the results of the join, and subsequent tabs displaying the new user (new hire) IDs, and terminated users.

## Create Example User List and HR Report

Here we'll create an example user list and HR report and export them to excel so that we can demonstrate how the script works. The example files are small, but they're enough to demonstrate how the script works.

First we'll need to import the pandas library. This will be the only library we need to import for the entire project.

In [1]:
# import pandas

import pandas as pd

Now we'll create first the user list. In this example it will be a simple user ID and user Name.

In [2]:
# create the user list and covert it to a dataframe

user_list_data= {
    'user ID': [1, 2, 3, 4],
    'user name':['John Smith', 'Jane Doe', 'Michael Lee', 'Sarah Brown']
}

user_list_df = pd.DataFrame(user_list_data)

user_list_df

,user ID,user name
0,1,John Smith
1,2,Jane Doe
2,3,Michael Lee
3,4,Sarah Brown


And now we'll create our HR report. In this example we'll have user ID and user name as well as other fields that often wont come with a user list, but are helpful for testing.

In [3]:
# create the HR report and convert it to a dataframe

hr_file_data = {
    'user ID': [1, 2, 3, 4, 5, 6],
    'user name': ['John Smith', 'Jane Doe', 'Michael Lee', 'Sarah Brown', 'Peter Parker', 'Laura Adams'],
    'job title': ['IT Specialist', 'HR Manager', 'Sales Executive', 'Accountant', 'Marketing Intern', 'Project Manager'],
    'department': ['IT', 'HR', 'Sales', 'Finance', 'Marketing', 'Operations'],
    'location': ['New York', 'Los Angeles', 'Chicago', 'Boston', 'San Francisco', 'Miami'],
    'manager': ['Mark Adams', 'Emily Lane', 'Lisa Green', 'John White', 'Emma Thompson', 'Robert Chen'],
    'hire date': ['2021-05-15', '2020-08-10', '2022-09-25', '2022-10-05', '2023-01-15', '2022-05-01'],
    'rehire date': ['', '', '', '', '', ''],
    'termination date': ['', '', '', '', '', '2023-06-15'],
    'employee status': ['Active', 'Active', 'Terminated', 'Active', 'Active', 'Terminated']
}

hr_file_df = pd.DataFrame(hr_file_data)

hr_file_df

,user ID,user name,job title,department,location,manager,hire date,rehire date,termination date,employee status
0,1,John Smith,IT Specialist,IT,New York,Mark Adams,2021-05-15,,,Active
1,2,Jane Doe,HR Manager,HR,Los Angeles,Emily Lane,2020-08-10,,,Active
2,3,Michael Lee,Sales Executive,Sales,Chicago,Lisa Green,2022-09-25,,,Terminated
3,4,Sarah Brown,Accountant,Finance,Boston,John White,2022-10-05,,,Active
4,5,Peter Parker,Marketing Intern,Marketing,San Francisco,Emma Thompson,2023-01-15,,,Active
5,6,Laura Adams,Project Manager,Operations,Miami,Robert Chen,2022-05-01,,2023-06-15,Terminated


Now we'll export user_list_df and hr_file_df to excel so that they can be used to demonstrate the script we're about to create.

In [4]:
user_list_df.to_excel('User List.xlsx', index=False)
hr_file_df.to_excel('HR File.xlsx', index=False)

## User Access Review Script

Now we'll write the user access review script, starting with importing the sample user list and HR report we created in the previous steps. At the end of this section we'll have an excel file which is comprised of the HR report joined to our user list so that we have expanded information for each of our users in the user list, as well as two helpful tabs displaying new users and terminated users found in our example user list.

First, we'll need to import the user list and hr file so that we can work with them.

In [5]:
# import the user list and hr report.

user_list = pd.read_excel('User List.xlsx')
hr_file = pd.read_excel('HR File.xlsx')

In [6]:
# Perform the join using user ID as the primary key since it is a unique value found between both files.

uar_file = pd.merge(user_list, hr_file, on='user ID', how='left')

uar_file

,user ID,user name_x,user name_y,job title,department,location,manager,hire date,rehire date,termination date,employee status
0,1,John Smith,John Smith,IT Specialist,IT,New York,Mark Adams,2021-05-15,NaN,NaN,Active
1,2,Jane Doe,Jane Doe,HR Manager,HR,Los Angeles,Emily Lane,2020-08-10,NaN,NaN,Active
2,3,Michael Lee,Michael Lee,Sales Executive,Sales,Chicago,Lisa Green,2022-09-25,NaN,NaN,Terminated
3,4,Sarah Brown,Sarah Brown,Accountant,Finance,Boston,John White,2022-10-05,NaN,NaN,Active


Now we'll create separate data frames for new users and terminated users. Later we'll use these to create tabs in our final Excel export.

To establish new users, assuming we don't have account creation dates, we'll rely on the hire date occurring sometime in the last year. We'll establish an arbitary date of October 1, 2022 as the cutoff. Anyone hired on or after this date will be considered a new user.

To determine terminated users, we'll say that anyone with an employee status of 'Terminated' is in fact a terminated user.

In [7]:
# create new hires dataframe

new_hires = uar_file[uar_file['hire date'] >= '2022-10-01']

new_hires

,user ID,user name_x,user name_y,job title,department,location,manager,hire date,rehire date,termination date,employee status
3,4,Sarah Brown,Sarah Brown,Accountant,Finance,Boston,John White,2022-10-05,NaN,NaN,Active


In [8]:
# create terminated users dataframe

terminated_users = uar_file[uar_file['employee status'] == 'Terminated']

terminated_users

,user ID,user name_x,user name_y,job title,department,location,manager,hire date,rehire date,termination date,employee status
2,3,Michael Lee,Michael Lee,Sales Executive,Sales,Chicago,Lisa Green,2022-09-25,NaN,NaN,Terminated


At this point we have all the elements needed to create our final Excel export. We will comebine them all justing the pandas ExcelWriter.

In [10]:
# create an Excel file called 'User Access Testing.xlsx' by combining the uar_file, new_hires, and terminated_users 
# dataframes.

with pd.ExcelWriter('User Access Testing.xlsx') as writer:
    uar_file.to_excel(writer, sheet_name='UAR File', index=False)
    new_hires.to_excel(writer, sheet_name='New Hires', index=False)
    terminated_users.to_excel(writer, sheet_name='Terminated Users', index=False)

## And there you have it!

We have a working user access testing script that can be used to quickly bring over additional user information as well as provide a ready to use Excel workbook with all of the results. 